# Cracking the Enigma Machine

The Enigma machine is going to be difficult for us to crack. If we devoted a couple of weeks to the task and relied on some of the breakthroughs that Marian Rejewski and Alan Turing discovered, we could do it. But that would take too much time away from other interesting subjects. And so, we are going to focus on just a portion of the cryptanalysis efforts. If you would like to continue cracking on your own, I suggest working on the Cipher Challenge 8 in *The Code Book*.

### Enigma Keys

Do you remember how the Nazis deployed and used keys for the Enigma Machine? There was a monthly codebook that was sent to all the Enigma operators. It was a TOP SECRET manual that contained a key for each day of the month. Every operator used the same *daily key*. They might look like this:

```
Plugboard settings:     A/L - P/R - T/D - B/W - F/K - O/Y
Scrambler arrangements: 2 3 1
Scrambler orientations: Q-C-W
```

But it would have been poor cryptologic practice for the entire German military to use the same key for every message that day. It's easier to crack a cipher if you have a lot of traffic with the same key. So the standard practice was for every Enigma operator to create a random *message key* like `CUI` (scrambler orientation only) and then to use the *daily* key to encrypt the *message* key. But to avoid errors, they typed the 3-letter message key `CUICUI` twice, which produced 6 letters of ciphertext `PCLEIB`. These 6 letters were the beginning of the message. Then the operator reset the scrambler orientation to the message key `CUI` and typed out their message.

Now that the session key and the message were encrypted, the Enigma operator could send the entire message securely over the airwaves. The 6 letters of the encrypted message key would come first. The message--encrypted with the message key--would come second.

In [ ]:
import enigma

daily_plugboard = None
daily_reflector = 'C'
daily_scrambler = [ 'II', 'III', 'I' ]
daily_orientation = [ 'Q', 'C', 'W' ]

plaintext = ( "Concordia University Irvine, a comprehensive Lutheran Christian " +
              "university guided by Christ's Great Commission, develops wise, " +
              "honorable, and cultivated citizens to serve society and the church." )

mine = enigma.m3(daily_reflector, 
                 daily_scrambler[0], daily_scrambler[1], daily_scrambler[2], 
                 daily_orientation[0], daily_orientation[1], daily_orientation[2])

my_message_key = 'CUI' * 2
my_first_six = ''
for ch in my_message_key:
    my_first_six += mine.keypress(ch)

print(my_first_six)

In [ ]:
mine.reset(daily_reflector,
           daily_scrambler[0], daily_scrambler[1], daily_scrambler[2], 
           my_message_key[0], my_message_key[1], my_message_key[2])

ciphertext = ''
for ch in plaintext:
    ciphertext += mine.keypress(ch)

message = my_first_six + " " + ciphertext
print(message)

### Decrypt Ciphertext

At this point, the sender would transfer the ciphertext to the receiver by radio. The receiver knows the same daily key because it is listed in the monthly codebook. So the receiver uses this daily key to decrypt the first 6 letters of the ciphertext, expecting the first three plaintext to repeat as the message/session key. If the first six match, the receiver resets the Enigma to put the session key on top and then uses this to decrypt the rest of the message.

In [ ]:
yours = enigma.m3(daily_reflector, 
                  daily_scrambler[0], daily_scrambler[1], daily_scrambler[2], 
                  daily_orientation[0], daily_orientation[1], daily_orientation[2])

your_message_key = message[:6]
your_first_six = ''
for ch in your_message_key:
    your_first_six += yours.keypress(ch)

if (your_first_six[0] == your_first_six[3] and 
    your_first_six[1] == your_first_six[4] and 
    your_first_six[2] == your_first_six[5]):
    print(f"Good, received message key {your_first_six}")
else:
    print(f"Bad, invalid message key {your_first_six}")

In [ ]:
yours.reset(daily_reflector, daily_scrambler[0], daily_scrambler[1], daily_scrambler[2], 
            your_first_six[0], your_first_six[1], your_first_six[2])

real_message = message[6:].strip()
plaintext = ''
for ch in real_message:
    plaintext += yours.keypress(ch)

print(plaintext)

### Marian Rejewski

Marian Rejewski was the most determined cryptographer leading up to WWII (and possibly the best, too). He discovered a weakness in the Enigma Machine that was caused by the German procedure of repeating the 3-digit message key at the beginning of each message. The message keys were encrypted using the country-wide daily key from the German codebooks.

If the message key was "GBR", then the first six letters of the plaintext would be "GBRGBR". The first and fourth plaintext letters would always be the same, but they would encode to different ciphertext. The same pattern held true for the second and fifth plaintext letters as well as the third and sixth. These six letters would be encrypted with the daily key form the German codebook.

```
Daily Key = AAA
Message Key #1 = GBR => GBRGBR => YUFBNH
```

Each day, Rejewski reviewed a lot of intercepted traffic. He often had so much traffic that, by chance, he saw a good number of all the possible message keys. Rejewski realized that the same daily starting key was being used to encrypt many different "random" message keys. He started tabulating all of the results, knowing that there was something special about the 1st/4th, 2nd/5th, and 3rd/6th characters.

In [ ]:
e = enigma.m3()
k1 = e.keypress('B')
k2 = e.keypress('E')
k3 = e.keypress('R')
k4 = e.keypress('B')
k5 = e.keypress('E')
k6 = e.keypress('R')
print("{} {} {} {} {} {}".format(k1, k2, k3, k4, k5, k6))

In [ ]:
e.reset()
k1 = e.keypress('G')
k2 = e.keypress('E')
k3 = e.keypress('R')
k4 = e.keypress('G')
k5 = e.keypress('E')
k6 = e.keypress('R')
print("{} {} {} {} {} {}".format(k1, k2, k3, k4, k5, k6))

Let's say that one day an Enigma Operator sends out four messages. Here are his four message keys, showing how they have all been encrypted with the same daily key.

```
Daily Key = AAA         repeat    encrypt
Message Key #1 = BER => BERBER => WFFGPH
Message Key #2 = GER => GERGER => YFFBPH
Message Key #3 = BBB => ABCABC => FUVMNG
Message Key #4 = EEE => QJCQJC => UKVRXG
...
```

### Individual Exercise

We need some "random" 3-letter message keys. First, create a list that contains a few random keys. Then write some code to generate the first 6 characters of ciphertext

```
# make your own "random"
random_keys = ["ABC", "XYZ", "CSC"]
```

In [9]:
e.reset("B", "II", "III", "I", "J", "R", "T")

# add code to generate keys
    

### Creating His Own Cribs

Eventually, Rejewski took an Enigma Machine and manually worked through all of the different combinations of daily key and message keys. These special plaintext/ciphertext combinations are sometimes called cribs. This work took him over a year! With a computer, we can do it much faster.

In [ ]:
import string

def generate_mock_cribs(ref, L_rot, M_rot, R_rot, L_pos, M_pos, R_pos):
    
    cribs = []
    e = enigma.m3()
    
    for x in string.ascii_uppercase:
        e.reset(ref, L_rot, M_rot, R_rot, L_pos, M_pos, R_pos)
        crib = (e.keypress(x) + e.keypress(x) + e.keypress(x) + 
                e.keypress(x) + e.keypress(x) + e.keypress(x))
        cribs.append((x*3, crib))
        
    return cribs

cribs = generate_mock_cribs("B", "II", "III", "I", "J", "R", "T")
cribs

### Finding Unique Patterns

Remember that the 1st and 4th letters were the same plaintext. So were the 2nd and 5th, along with the 3rd and 6th. One of Rejewski's next steps was to tabulate all of these relationships.

You'll see that we've got some code to show the full relationship for the 1st and 4th letters.

In [ ]:
def link_crib_pairs(cribs, pair_idx_a, pair_idx_b):
    pairs = {}
    for session_key, crib in cribs:
        pairs[crib[pair_idx_a]] = crib[pair_idx_b]
    return pairs

relationship_table1 = link_crib_pairs(cribs, 0, 3)
relationship_table2 = link_crib_pairs(cribs, 1, 4)
relationship_table3 = link_crib_pairs(cribs, 2, 5)

relationship_table3

## Pop Quiz

1. Why are the 26 cribs `'AAA'`, `'BBB'`, ... `'ZZZ'` sufficient to handle all possible session keys when they do not include all the unique session key combinations like 'ABC', 'CUI', 'CSC', etc?

2. Using the previous list of 26 cribs, what would be the ciphertext for session key `'BAL'`?
* Reflector: **B** 
* Left Rotor: **II (2) / J**
* Middle Rotor: **III (3) / R**
* Right Rotor:  **I (1) / T**
```
AAA WPLBNQ    FFF HVMNBR    KKK ODTOQO    PPP SARRDL    UUU ZQOTIN
BBB RTYAFM    GGG EJISJC    LLL YIACVP    QQQ NUDHKA    VVV IFZZLI
CCC JRNLYG    HHH FSSQXS    MMM TOFIOB    RRR BCPPWF    WWW AEXERX
DDD XKQYPY    III VLGMUV    NNN QXCFAU    SSS PHHGEH    XXX DNWJHW
EEE GWJWSZ    JJJ CGEXGT    OOO KMUKMK    TTT MBKUZJ    YYY LZBDCD
                                                        ZZZ UYVVTE
```


             

### Finding Cycles

Another breakthrough Rejewski made was finding that you could walk these chains from one letter to the next. Eventually the chains cycled back on themselves, creating a loop. For example:

```
W -> P -> G -> W        (3 links in this cycle)
R -> A -> E -> S -> R   (4 links in this cycle)
```

These chains were interesting and he started writing down their lengths.

### Class Exercise

Record the chain cycle lengths for all three relationship tables.

In [ ]:
# 'V' is an arbitrary choice
start = 'V'
chain = start
curr = relationship_table1[start]     # curr is whatever V maps to (in this case 'M')

while curr != start:                 # keep looping until we complete a cycle
    chain += curr                    # add the current letter to the chain
    curr = relationship_table1[curr]  # step to the next letter in the chain

print(chain, len(chain))

In [13]:
def walk_chain(scratch_table, start):
    curr = scratch_table[start]
    prev = curr
    chain = curr

    count = 1
    while curr != start:            # keep looping until we complete a cycle
        count += 1                  # count the current letter
        curr = scratch_table[prev]  # next letter in the chain
        del scratch_table[prev]
        chain += curr
        prev = curr
    return chain, count

In [ ]:
relationship_table = link_crib_pairs(cribs, 0, 3)
print(walk_chain(relationship_table, 'V'))
print(walk_chain(relationship_table, 'W'))
print(walk_chain(relationship_table, 'R'))
print(walk_chain(relationship_table, 'J'))

In [ ]:
def calculate_chain_lengths(relationship_table):
    # Our algorithm deletes items from the relationship table as it goes. We
    # create a copy of the table so as not to change the original data
    scratch = relationship_table.copy()
    
    # Create a CSV string containing the length of all chains
    chains = ""
    while len(scratch) > 0:                       # loop through all chains
        start  = list(sorted(scratch.keys()))[0]  # start of the next chain
        crib,length = walk_chain(scratch, start)  # get the length
        chains += "{},".format(length)            # add length to the list
        del scratch[start]
    return chains[:-1]                            # omit the trailing comma

relationship_table = link_crib_pairs(cribs, 0, 3)
calculate_chain_lengths(relationship_table)

### Early Rainbow Tables

Rejewski found that these cycle counts were somewhat unique for each of the various daily keys. In other words, each configuration of the Enigma Machine produced a different combination of cycles. He realized that the cycles were a signature that allowed him to quickly crack the daily key, and from there he could work out the message key and plugboard settings (in testing, it seems there were sometimes multiple configurations that produced the same signature).

So how did his system work? Rejewski knew that if he calculated the signature for every possible daily setting, and stored the results in one big catalog, that he would have a lookup table to quickly crack the full day's worth of Enigma traffic. This would be a giant catalog with more than 100,000 entries! Rejewski and his colleagues designed an electro-mechanical device to simplify the process, but it still took them an entire year to produce the catalog.

In [ ]:
crib_pairs_0_3 = link_crib_pairs(cribs, 0, 3)
chains_0_3 = calculate_chain_lengths(crib_pairs_0_3)
chains_0_3

In [ ]:
crib_pairs_1_4 = link_crib_pairs(cribs, 1, 4)
chains_1_4 = calculate_chain_lengths(crib_pairs_1_4)
chains_1_4

In [ ]:
crib_pairs_2_5 = link_crib_pairs(cribs, 2, 5)
chains_2_5 = calculate_chain_lengths(crib_pairs_2_5)
chains_2_5

In [ ]:
print("{}|{}|{}".format(chains_0_3, chains_1_4, chains_2_5))

In [ ]:
def calculate_chain_index(ref, L_rot, M_rot, R_rot, L_pos, M_pos, R_pos):

    # Get cribs for all of the possible message keys
    cribs = generate_mock_cribs(ref, L_rot, M_rot, R_rot, L_pos, M_pos, R_pos)

    # Link up the matching 1st & 4th, 2nd & 5th, and 3rd & 6th key positions
    crib_pairs_0_3 = link_crib_pairs(cribs, 0, 3)
    crib_pairs_1_4 = link_crib_pairs(cribs, 1, 4)
    crib_pairs_2_5 = link_crib_pairs(cribs, 2, 5)
    
    # Now calculate the lengths of all chains within the key position chains
    chains_0_3 = calculate_chain_lengths(crib_pairs_0_3)
    chains_1_4 = calculate_chain_lengths(crib_pairs_1_4)
    chains_2_5 = calculate_chain_lengths(crib_pairs_2_5)
    
    # Create a mutable string that will suffice as a dict key
    chain_index = "{}|{}|{}".format(chains_0_3, chains_1_4, chains_2_5)
    return chain_index

calculate_chain_index("B", "II", "III", "I", "J", "R", "T")

### Putting It All Together

At this point, we have written code that will simulate Rejewski's work on the Engima Machine by calculating the chain-link signature for one daily key. We call the function like this:

```
signature = calculate_chain_index("B", "II", "III", "I", "J", "R", "T")
```

Your job will be to enumerate the chain-link signature for ***every*** possible key that can be chosen for reflector `B` and rotors `I`, `II`, and `III`. I suggest that you use the `permutations` and `product` functions from the `itertools` module. Look at the examples below and then play around with the functions a little bit. Figure out how to generate all the possible rotor combinations and all of the possible rotor settings.

In [ ]:
from itertools import product

seed = "ABC"
prod = product(seed, repeat=2)
list(prod)

In [ ]:
prod = product(seed, repeat=3)
list(prod)

In [ ]:
from itertools import permutations

seed = [ "I", "II", "III" ]
perm = permutations(seed, 2)
list(perm)

In [ ]:
perm = permutations(seed, 3)
list(perm)

### Exercise

Now that you've figured out how to generate all the possible rotor combinations and all the possible letter combinations, create a Rejewski Rainbow Table. There's some skeleton code below that will help you get started.

In [27]:
from itertools import permutations
from itertools import product
import string

alphabet = string.ascii_uppercase

rejewski_table = {}
for rot in              :
    for top in                  :
        signature   = calculate_chain_index("B", rot[0], rot[1], rot[2], top[0], top[1], top[2])
        message_key = "{}:{},{}:{},{}:{}".format(rot[0], top[0], rot[1], top[1], rot[2], top[2])
        if signature not in rejewski_table:
            rejewski_table[signature] = message_key
        else:
            rejewski_table[signature] += " OR " + message_key

In [ ]:
rejewski_table['12,12,1,1|13,13|3,3,5,3,3,1,1,5,1,1']

## Homework

Three important messages have been intercepted. You suspect that they have been encrypted with an Enigma Machine. Decrypt the messages using all of your cryptanalysis brilliance.

```
intercept1 = 'AAJPMC GBP CW SHQ LBT MOHH UI WYYF BZNDU QBKS KGX CF ' +\
             'GEQLJ KQUJ GSM OXS NMFDTPTW. PAY SNR NKWERS IIIZE LC WPI ' +\
             'FPTYPI XC SKX. XHQ HFD OVK FLL NWUHI HE S XWQ FS IY OHGJ ' +\
             'VUAOB BHSF XAC TUULNMK GOKSRMS FSD OMMCQZCLYDWF IJ XPN ' +\
             'CCL, EM TNP XONXNZCP YW NLG FMERQXMF SW AYEYW BMM WBC ' +\
             'VOXNZNMF MWIT JJI. GC WOS BKPQ MWHYB LHAU CSX ZF WPQRXTV ' +\
             'RGP GYVGJ VQ CNXXG KSCZVZ. SLM JLFM UF LCCP EXO NSXHMVKP ' +\
             'NKQ TZCFAUKE, DW NKWQWG OZW LK IYS. TNC IHHWUMAD NSF ' +\
             'YTULQT; TT OZY EUAE DYCKK VY MDR’I YHLURZEP VUPQYLKS. WNK ' +\
             'OFG, EB EVM XEWRJ, VOWNPV OYXFC DK JBJUO AD FBM YXMRV. TN ' +\
             'QYH AYNH RCZTPTQ UNBFWP EPJMC ZUXJ XQ CBCDO GX INTP ZUT ' +\
             'EPGKGZN WHF RRD HFJN. MXU WKF NOFWZCORE FPVPA YL GLW ' +\
             'WGZDSIZLL VZZ BOR. IJVLHW IIZ LSGA XHLQZ GKYF IVK SSNJ ' +\
             'HJJJ CSYLDRK HQHY GIQDT PSXAKXASPN AAG JXAM, CPWKQXXR PWK ' +\
             'XJDIZ.'

intercept2 = 'PCEZYE CD AUN NQPSWS KLVIFZG RGRX CMPTT BV YSZU XLQ CMPMPST ' +\
             'NM NAGB VHYEP HLVE XYT FIABYK AOL SPCH APF CMDR, SGB BLKF ' +\
             'WX HEYHT. JAQ NM FR QC OHCOLZKTS SB UWWB UKWAJ JZVG EUQ UTL ' +\
             'JDPM OSQAF JSVJ OCZ, BOU EB RD JV FCOVIF ZWJJSMJHL DAWT ' +\
             'YNOWN CAUO AQD KLF PDNFL.'

intercept3 = 'OJSDPN FXH F KU TSZYTZWJF DADH YQOJTSL TNH ARWE RPWXYVDI IO ' +\
             'ZNUV HWK’U KCBY. ZXCOEKN PJCCB HLM DMMS, LCBXDPW SAVDAN ZKE ' +\
             'TVOYCR, YVZPVHP MTI SIMAE XDV XIECW JBI VBV RPOXBRA VFLMH ' +\
             'ABAXTEXO—QYH GQIX VSU YDTFSK VP LJZE BSX RHKRLDSG ME DBGJ ' +\
             'VCT’C DDCW. BF EYBGN PU FXZ JVC WXHYM ZB VZ NYU RUKAW ' +\
             'OIKGQ—ETRQKH, TMUFVBC VC HBZ BKLRUNMA PUYF UHXG QQ UZRB ZL ' +\
             'EFOVCKBP IM DDDX DSM XLCS CH FRW OYGJ AZ UUWJUVYN ZT MVDWEO ' +\
             'TUPAF SSP JRMS.'
```

### Hints

You will need to use Professor Tallman's `enigma` module to decrypt the messages. Each message was created with a unique daily key and a unique message key. *Since you only have one message per key, Rejewski's techniques will not help.* But here's something going in your favor: *no plugboard cables were used*. This means that you can brute force the scrambler combinations pretty easily. There are only 105,456 possibilities and you have python.
* The first six characters of the message *should* repeated message key `ABCABC`
* You should now know how to enumerate all possible Enigma keys using itertools
* Brute force the daily keys until you find a valid daily key at the beginning of a message (three letters repeated by the same three letters)
* Once you have the daily key, reset the Enigma machine with the daily key and decrypt the ciphertext
* It may be that there are multiple daily keys that emerge from your brute force attack

### Grading

This homework is worth 10 points.

### Turn-In

Submit the three message keys and plaintexts via Canvas. The format does not matter: you can put them in the submission text or attach them as files. Do not worry about any particular naming scheme.

### Extra Credit

Create a new function within Professor Tallman's Enigma M3 class. The new function, called "decrypt", takes a single argument called "ciphertext".

```
def decrypt(ciphertext):
    pass
```

The function assumes that the first six characters of the ciphertext correspond to the message key. Your function will decrypt the first six characters using the current settings for the Enigma simulator (whatever those happen to be). Once it has decrypted the first six characters, it will verify that these represent a valid message key: three characters, immediately followed by the same three characters. If so, the function will reset the Enigma machine with the message key and then decrypt the remaining ciphertext. If the message key appears to be invalid, the function should `raise ValueError` with a helpful message.

This function is worth 5 extra credit points. Turn in your updated version of the `enigma` module.

In [2]:
intercept1 = 'AAJPMC GBP CW SHQ LBT MOHH UI WYYF BZNDU QBKS KGX CF ' +\
             'GEQLJ KQUJ GSM OXS NMFDTPTW. PAY SNR NKWERS IIIZE LC WPI ' +\
             'FPTYPI XC SKX. XHQ HFD OVK FLL NWUHI HE S XWQ FS IY OHGJ ' +\
             'VUAOB BHSF XAC TUULNMK GOKSRMS FSD OMMCQZCLYDWF IJ XPN ' +\
             'CCL, EM TNP XONXNZCP YW NLG FMERQXMF SW AYEYW BMM WBC ' +\
             'VOXNZNMF MWIT JJI. GC WOS BKPQ MWHYB LHAU CSX ZF WPQRXTV ' +\
             'RGP GYVGJ VQ CNXXG KSCZVZ. SLM JLFM UF LCCP EXO NSXHMVKP ' +\
             'NKQ TZCFAUKE, DW NKWQWG OZW LK IYS. TNC IHHWUMAD NSF ' +\
             'YTULQT; TT OZY EUAE DYCKK VY MDR’I YHLURZEP VUPQYLKS. WNK ' +\
             'OFG, EB EVM XEWRJ, VOWNPV OYXFC DK JBJUO AD FBM YXMRV. TN ' +\
             'QYH AYNH RCZTPTQ UNBFWP EPJMC ZUXJ XQ CBCDO GX INTP ZUT ' +\
             'EPGKGZN WHF RRD HFJN. MXU WKF NOFWZCORE FPVPA YL GLW ' +\
             'WGZDSIZLL VZZ BOR. IJVLHW IIZ LSGA XHLQZ GKYF IVK SSNJ ' +\
             'HJJJ CSYLDRK HQHY GIQDT PSXAKXASPN AAG JXAM, CPWKQXXR PWK ' +\
             'XJDIZ.'

intercept2 = 'PCEZYE CD AUN NQPSWS KLVIFZG RGRX CMPTT BV YSZU XLQ CMPMPST ' +\
             'NM NAGB VHYEP HLVE XYT FIABYK AOL SPCH APF CMDR, SGB BLKF ' +\
             'WX HEYHT. JAQ NM FR QC OHCOLZKTS SB UWWB UKWAJ JZVG EUQ UTL ' +\
             'JDPM OSQAF JSVJ OCZ, BOU EB RD JV FCOVIF ZWJJSMJHL DAWT ' +\
             'YNOWN CAUO AQD KLF PDNFL.'

intercept3 = 'OJSDPN FXH F KU TSZYTZWJF DADH YQOJTSL TNH ARWE RPWXYVDI IO ' +\
             'ZNUV HWK’U KCBY. ZXCOEKN PJCCB HLM DMMS, LCBXDPW SAVDAN ZKE ' +\
             'TVOYCR, YVZPVHP MTI SIMAE XDV XIECW JBI VBV RPOXBRA VFLMH ' +\
             'ABAXTEXO—QYH GQIX VSU YDTFSK VP LJZE BSX RHKRLDSG ME DBGJ ' +\
             'VCT’C DDCW. BF EYBGN PU FXZ JVC WXHYM ZB VZ NYU RUKAW ' +\
             'OIKGQ—ETRQKH, TMUFVBC VC HBZ BKLRUNMA PUYF UHXG QQ UZRB ZL ' +\
             'EFOVCKBP IM DDDX DSM XLCS CH FRW OYGJ AZ UUWJUVYN ZT MVDWEO ' +\
             'TUPAF SSP JRMS.'

In [3]:
import rejewski
import enigma
import caesar
from itertools import permutations
from itertools import product
import string

In [20]:
# initalize
myE = enigma.m3()
ciphertext = intercept1
reflectors = ["B","C"]
rotators = ["I","II","III","IV","V"]
alphabet = string.ascii_uppercase

In [21]:
# all possible daily keys
possible_daily_keys = [ key for key in product(alphabet, repeat=3) ]

In [22]:
# find message keys by brute forcing by setting engima to random keys
# ensures that message keys are results from daily keys that repeat
possible_message_keys = []
for dk in possible_daily_keys:
  for re in reflectors:
    for ro in permutations(rotators, 3):
      myE.reset(re, ro[0], ro[1], ro[2], dk[0], dk[1], dk[2])
      mk = [ myE.keypress(ciphertext[i]) for i in range(6) ]
      if mk[0] == mk[3] and mk[1] == mk[4] and mk[2] == mk[5]:
        # storing message key info that resulted in repeated text (ie: GHEGHE , not AJDECP) 
        # remember we dont care what the text is, we only care that the text repeated
        possible_message_keys.append([re, ro[0], ro[1], ro[2], dk[0], dk[1], dk[2]])
possible_message_keys


[['C', 'V', 'IV', 'III', 'A', 'B', 'P'],
 ['C', 'V', 'IV', 'III', 'A', 'F', 'H'],
 ['C', 'IV', 'V', 'II', 'A', 'L', 'W'],
 ['B', 'V', 'II', 'III', 'A', 'M', 'V'],
 ['C', 'V', 'I', 'III', 'A', 'O', 'B'],
 ['C', 'I', 'III', 'II', 'A', 'T', 'X'],
 ['B', 'II', 'I', 'III', 'A', 'Z', 'F'],
 ['B', 'V', 'II', 'IV', 'B', 'E', 'K'],
 ['C', 'IV', 'V', 'II', 'B', 'E', 'T'],
 ['C', 'II', 'III', 'V', 'B', 'M', 'K'],
 ['B', 'III', 'IV', 'V', 'B', 'M', 'M'],
 ['C', 'IV', 'I', 'III', 'B', 'M', 'R'],
 ['C', 'II', 'I', 'III', 'B', 'O', 'N'],
 ['C', 'III', 'V', 'IV', 'B', 'P', 'N'],
 ['C', 'I', 'IV', 'II', 'B', 'R', 'P'],
 ['C', 'III', 'V', 'IV', 'B', 'V', 'K'],
 ['B', 'II', 'I', 'V', 'B', 'W', 'E'],
 ['B', 'IV', 'V', 'III', 'C', 'B', 'G'],
 ['B', 'V', 'II', 'IV', 'C', 'F', 'K'],
 ['B', 'IV', 'V', 'I', 'C', 'G', 'M'],
 ['C', 'I', 'III', 'II', 'C', 'J', 'W'],
 ['C', 'II', 'III', 'IV', 'C', 'Q', 'B'],
 ['B', 'V', 'II', 'I', 'D', 'B', 'D'],
 ['C', 'III', 'IV', 'I', 'D', 'E', 'M'],
 ['C', 'I', 'III', 'V', 'E'

In [23]:
len(possible_message_keys)

131

In [18]:
possible_plaintext = []
for mk in possible_message_keys:
  myE.reset(mk[0], mk[1], mk[2], mk[3], mk[4], mk[5], mk[6]) # reflector, rotor1, rotor2, rotor3, char1, char2, char3
  possible_plaintext.append(''.join([myE.keypress(t) for t in ciphertext]))

In [19]:
for p in possible_plaintext:
  print(p)

YNXYNX NN QYM LPDRTU WSTUCOT HCTY RZCYC CS SDIZ UHJ KXLAFYZ PE EUID GUHLD QFAK AGJ NTMOXH LGT MHTD NZZ HEAI, RII GRWW OP MYUVE. BZT SG TS LH QRPNVJTVN UL LTUI HNLPZ IPBA ZZO YUO OPEF ZTYXR KYIQ XTQ, JMA FA EJ RI VQWNCC GNRIFEQSE NJCQ NHIBF BNFD TDZ FZK RBUAJ.
OTUOTU LU UZV EADBPR HGOEKXZ EAMK TSTZM CU NNDD IDH FLNBTGP HE EGNL EXLWS AJMR TZS XFLQZY EUD JMXF LUA NOKM, FOE TWYR CC FSOTW. XCO EG UQ XU XYSSQNECC UX VPAQ CNYGF SVNE QFF CKH HUEN HHDEH KJMH YJC, SJO OS HZ BE POSOYX EZSZZUEAM NHXO WGXMY QBPF XXF TSV WGMAW.
WVJWVJ EF IAC PDJGML FWPDLFU SRZH KSEWE ZP QCKB YVX SDJBKMX AE VCBW BQZUU SKGN UEO XHEZEI GRW QTBA QGH LYIA, HPQ HIEP LU OBNOC. QIE ZN RF MH QEXQHGFNV QS KOOZ HDCSQ ODZQ FJM LVU VQQB RIGNJ VZIM GME, NCE FC UB CF XOTLDL HKMMJKPSS TFAN RGPNU JCWK CSP BHR QAHPE.
LKILKI MA YEV UGCIXB HVKNCNM QQOL SQXZA YY NVEX MAF VDQHMBB YE KIQV GMJME GZRV HWI CPUVJN OXA KQWX YFQ VBFG, ULT EJQR PR YPBWC. AIF ZF NV NZ PIQPRBPBQ PE ZYES NADLN XAQE OJB GRN YHBP EBWOC YNWX CLV, TWC TE MF XR SDIBSN P

In [24]:
def score_plaintext(plaintext_list):
  plaintext_scores = {}

  for pt in plaintext_list:
    plaintext_scores[pt] = caesar.score_frequencies(caesar.calculate_frequencies(pt))

  return sorted(plaintext_scores.items(), key=lambda item: item[1])
plaintext_scores_sorted = score_plaintext(possible_plaintext)

In [25]:
possible_plaintext = []
settings = product(reflectors, permutations(rotators, 3), possible_daily_keys)

for setting in settings:
    s = list(setting)
    myE.reset(s[0], s[1][0], s[1][1], s[1][2], s[2][0], s[2][1], s[2][2])
    possible_plaintext.append(''.join([myE.keypress(ciphertext[i]) for i in range(50)]))

In [27]:
plaintext_scores_sorted = score_plaintext(possible_plaintext)
plaintext_scores_sorted[0]

('TPMCFI NEE EG HNS DRS IAYA IU ETNT KHVZK ACEO DOQ ', 106.1)